In [1]:
from pathlib import Path
import pandas as pd
import boto3
from io import StringIO




In [2]:
client = boto3.client('s3')
bucket_name = 'columbia-project-resources'
object_key = 'bitcoin.csv'
csv_obj = client.get_object(Bucket=bucket_name, Key=object_key)
body = csv_obj['Body']
csv_string = body.read().decode('utf-8')

bitcoin_df = pd.read_csv(StringIO(csv_string))
bitcoin_df.head()

NoCredentialsError: Unable to locate credentials

#module 11 - Evelyn